In [17]:
import json
import requests

In [22]:
database_name = "WPE_test_2022"
mongo_server_name = "mongodb://192.168.135.21:27017/"
collection_name = "Posts"
crawleres_manager_url = "http://cm.portal.aps/api/v2/crawl"

In [23]:
def delete_query(crawl_id: str):
    url = f"http://cm.portal.aps/api/v2/crawl/{crawl_id}?force=true"
    headers = {"Content-Type": "application/json"}
    response = requests.request("DELETE", url, headers=headers)
    print(response)

In [24]:
objects =["https://www.kommersant.ru/doc/5292181"]

Создадим сервис для извлечения текста новости

Для этого создадим сервис с соответствующими параметрами

In [25]:
payload = json.dumps({
  "workflow_id": 111111111111111,
  "theme": 0,
  "run_after": [],
  "sntag": "nws",
  "mongo_server_name": mongo_server_name,
  "mongo_database_name": database_name,
  "mongo_collection_name": collection_name,
  "app_settings": {
    "docker_name": "dreg.apsolutions.ru/news_crawler_v2",
    "workers_count": 1,
    "image_tag": "latest"
  }
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", crawleres_manager_url + "/NWS" , headers=headers, data=payload)

print(response.text)

db_crawl_id = json.loads(response.text)["crawl_id"]

print(db_crawl_id)

ConnectionError: HTTPConnectionPool(host='cm.portal.aps', port=80): Max retries exceeded with url: /api/v2/crawl/NWS (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F3920F0D30>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

Закидываем в созданную очередь задачи на обработку

In [6]:
url = f"http://cm.portal.aps/api/v2/crawl/{db_crawl_id}"

payload = json.dumps({
    "mongo_server_name": mongo_server_name,
    "mongo_database_name": database_name,
    "mongo_collection_name": collection_name,
    "workflow_id": "1644949782675",
    "run_after": [],
    "task_fields_json": "{\n  \"WorkersCount\": 1,\n  \"download_files\": 0,\n  \"on_diffbot_failure\": \"skip\",\n  \"InDataType\": \"URL\",\n  \"OutCollection\": \"Posts\"\n}",
    "entity_type": "URL",
    "sntag": "NWS",
    "theme": 159,
    "main_db_server": "mongodb://192.168.135.21:27017",
    "main_db": "MIP_Global",
    "priority": 5,
    "use_proxy": False,
    "crawl_full_pages": False,
    "download_depth": 1,
    "url_address": "",
    "dict_db": "MIP_Global",
    "dict_server": "mongodb://192.168.135.21:27017",
    "user": "",
    "min_date": 0,
    "sn_tag": "NWS",
    "crawl_id": db_crawl_id,
    "objects": objects,
    "task_id": "cffe8b09-cc40-4b34-9915-bea5651611aa"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("PUT", url, headers=headers, data=payload)

print(response.text)
print(response.status_code)


NameError: name 'db_crawl_id' is not defined

In [523]:
delete_query(db_crawl_id)

<Response [204]>


In [574]:
print('db.getCollection("Posts").find({CrawlId :"',db_crawl_id,'"}).sort({_id: -1})', sep='')

db.getCollection("Posts").find({CrawlId :"NWS-1648842275811448"}).sort({_id: -1})


Прогресс выполнения можно посмотреть по crawl_id [тут](http://192.168.135.21:15672/#/queues)

```
test
test
```

А результаты работы доступны в коллекции collection базы данныъ database_name сервера mongo_server_name. Для просмотра можно воспользоваться [robo3t](https://robomongo.org/)

```
database_name = "WPE_test_2022"
mongo_server_name = "mongodb://192.168.135.21:27017/"
collection = "Posts"
```

Пример запроса:
```
db.getCollection('Posts').find({CrawlId : "NWS-1645209213614780"}).sort({_id: -1})
```
где `{CrawlId : "NWS-1645209213614780"}` - условие поиска (найти все документы, собранные сервисом с указанным crawl_id ),
`.sort({_id: -1}` - упорядочить результаты по дате загрузки в базу. Тоже самое можно сделать из питона.

In [577]:
from pymongo import MongoClient
client = MongoClient(mongo_server_name)
db = client[database_name]
posts = db[collection_name]

In [578]:
cursor = posts.find({'CrawlId' : db_crawl_id})
for post in cursor:
    print(post)

{'_id': ObjectId('624742f2a9ceb108ebeddd9a'), 'ID': '503eda8d6152e5daa55f594c0d235d12936e401f602720c5d26b99675c98a947', 'ItemType': 1, 'LoadDate': datetime.datetime(2022, 4, 1, 18, 22, 33, 434000), 'CrawlId': ['NWS-1648837349503130', 'NWS-1648839829369628', 'WPE-1648839868048799', 'NWS-1648842099823798', 'NWS-1648842275811448'], 'WorkflowId': [111111111111111, 333], 'CrawlerType': 2, 'Title': '«Газпром» потерял контроль над европейскими ПХГ и трейдингом', 'Text': '«Газпром» утратил контроль над компанией – оператором подземных хранилищ газа (ПХГ) Германии «Астора» (astora), следует из сообщения газовой монополии, обнародованного 1 апреля. В релизе поясняется, что 31 марта дочерние структуры «Газпрома» вышли из капитала компании «Газпром экспорт бизнес–сервисы», являющейся единственным владельцем Gazprom Germania GmbH. Кто стал новым держателем, в сообщении не указывается.\nGazprom Germania GmbH включает в себя компанию–дистрибутора WINGAS/WIEE – оператора ПХГ «Астора». Кроме того, в со

По окончании работы, не забываем убить за собой сервис

In [579]:
delete_query(db_crawl_id)

<Response [204]>


Теперь соберем WPD этих же страниц. Для этого возьмем пару доп. функций

In [580]:
def create_query(workers_count: int, database_name, docker_name, sntag, pas_resourse_id="vk"):
    url = f"http://cm.portal.aps/api/v2/crawl/{sntag}"

    payload = {
        "mongo_server_name": mongo_server_name,
        "mongo_database_name": database_name,
        "mongo_collection_name": collection_name,
        "workflow_id": 333,
        "task_type": 3,
        "app_settings": {"workers_count": workers_count},
        "options": {"proxy": 1, "fields": ["root_element", "screenshot"]},
    }

    payload["app_settings"]["docker_name"] = docker_name
    headers = {"Content-Type": "application/json"}
    response = requests.request("POST", url, json=payload)
    crawl_id = json.loads(response.text)["crawl_id"]
    return crawl_id


def send_payload(crawl_id, payload):
    url = f"http://cm.portal.aps/api/v2/crawl/{crawl_id}"
    headers = {"Content-Type": "application/json"}
    response = requests.request("PUT", url, headers=headers, json=payload)
    print(response)
    
    
# def send_data_webpage_extractor(
#     crawl_id,
#     objects,
#     task_type,
#     database_name,
#     collection,
#     task_fields_json={},
#     resource="vki",
# ):
    

In [581]:
# создаем рабочий процесс прокачки web page dump
wpd_crawl_id = create_query(
    workers_count=1,
    database_name=database_name,
    docker_name="dreg.apsolutions.ru/webpage_extractor_v2",
    sntag="WPE",
    pas_resourse_id="vk",
)

print(wpd_crawl_id)

WPE-1648842306094214


Кидаем в процесс задачу

In [582]:
#objects = ["https://www.rbc.ru/society/18/02/2022/620fd7409a7947aaa25d8377?from=from_main_5"]
#objects = ["https://ura.news/news/1052542740"]

url = f"http://cm.portal.aps/api/v2/crawl/{wpd_crawl_id}"
headers = {"Content-Type": "application/json"}
task_fields_json = {
    "mongo_server_name": mongo_server_name,
    "mongo_database_name": database_name,
    "mongo_collection_name": collection_name,
    "workflow_id": 333,
    "task_type": 3,
    "pas_resource": "vki",
    "fields": ["root_element", "screenshot"],
    "objects": objects,
    "app_settings": {
        "workers_count": 1
    },
}

payload = {"task_fields_json": json.dumps(task_fields_json), "kek":"kek", "fields": ["root_element", "screenshot"], "objects": objects}

response = requests.request("PUT", url, headers=headers, json=payload)
print(response)

<Response [204]>


In [583]:
objects

['https://www.vedomosti.ru/business/articles/2022/04/01/916299-gazprom-poteryal-kontrol-evropeiskimi-phg']

Давайте найдем наш материал в базе:
```
db.getCollection('Posts').find({URL: "https://www.vedomosti.ru/business/articles/2022/04/01/916299-gazprom-poteryal-kontrol-evropeiskimi-phg"})
```

Видим, что по итогам сбора у нас 1) Добавился еще один crawl_id 2) Появилась секция WebPageDumps

Не забываем убить не нужный нам более обработчик

In [584]:
delete_query(wpd_crawl_id)

<Response [204]>


Из полученного дампа можно получить структуру страницы, запустив еще один обработчик

In [601]:
payload = json.dumps({
  "workflow_id": 111111111111111,
  "theme": 0,
  "run_after": [],
  "sntag": "WSP",
  "mongo_server_name": mongo_server_name,
  "mongo_database_name": database_name,
  "mongo_collection_name": collection_name,
  "app_settings": {
    "docker_name": "dreg.apsolutions.ru/wsp_parser_v2",
    "workers_count": 0,
    "image_tag": "latest"
  }
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", crawleres_manager_url + "/WSP" , headers=headers, data=payload)

print(response.text)

wsp_crawl_id = json.loads(response.text)["crawl_id"]

print(wsp_crawl_id)

{"crawl_id":"WSP-1648843103342162","workflow_id":"111111111111111","url":"http:\/\/cm.portal.aps\/api\/v2\/crawl\/WSP-1648843103342162","input_url":"http:\/\/cm.portal.aps\/api\/v2\/crawl\/WSP-1648843103342162\/input","errors_url":"http:\/\/cm.portal.aps\/api\/v2\/crawl\/WSP-1648843103342162\/errors"}
WSP-1648843103342162


In [604]:
url = f"http://cm.portal.aps/api/v2/crawl/{wsp_crawl_id}"
#objects = ["https://ura.news/news/1052542740"]

payload = json.dumps({
    "mongo_server_name": mongo_server_name,
    "mongo_database_name": database_name,
    "mongo_collection_name": collection_name,
    "workflow_id": "1644949782675",
    "run_after": [],
    "task_fields_json": "{  \"WorkersCount\": 0, \"mongo_server_name\": \"mongodb://192.168.135.21:27017/\", \"mongo_database_name\": \"WPE_test_2022\",\"mongo_input_collection_name\": \"Posts\",\"mongo_output_collection_name\": \"Blocks\",\"mode\": \"initial\"}",
    "objects": objects,
    "app_settings": {
        "workers_count": 0
    }
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("PUT", url, headers=headers, data=payload)
print(response.status_code)
print(response.text)


204



In [605]:
url = f"http://cm.portal.aps/api/v2/crawl/{wsp_crawl_id}"
#objects = ["https://ura.news/news/1052542740"]

payload = json.dumps({
    "mongo_server_name": mongo_server_name,
    "mongo_database_name": database_name,
    "mongo_collection_name": collection_name,
    "workflow_id": "1644949782675",
    "run_after": [],
    "task_fields_json": "{  \"WorkersCount\": 0, \"mongo_server_name\": \"mongodb://192.168.135.21:27017/\", \"mongo_database_name\": \"WPE_test_2022\",\"mongo_input_collection_name\": \"Posts\",\"mongo_output_collection_name\": \"Blocks\",\"mode\": \"blocks_extraction\"}",
    "objects": objects,
    "app_settings": {
        "workers_count": 1
    }
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("PUT", url, headers=headers, data=payload)

print(response.text)
print(response.status_code)



204


In [595]:
delete_query(wsp_crawl_id)

<Response [204]>


In [598]:
url = objects[0]

In [599]:
import gridfs
grid_fs = gridfs.GridFS(db)

In [600]:
post = posts.find_one({"URL": url})

wpd_filename = post["WebPageDumps"][-1]["Desc"]
wpd_file = grid_fs.find_one({"filename": wpd_filename})

In [157]:
!pip install webpagelib

ERROR: Could not find a version that satisfies the requirement webpagelib (from versions: none)
ERROR: No matching distribution found for webpagelib


In [ ]:
import webpagelib
wpd_data = webpagelib.WebPageDump.from_messagepack(wpd_file.read())

In [ ]:
!pwd

In [ ]:
import parser

def parse(wpd, url):
    features = {
        "Blocks": {},
        "Text": "",
        "MatchingBlocks": [],
        "Grid": [],
        "Image": {},
        "URL": url,
    }
    image_size, features["Blocks"] = parser.extract_data_from_WPD(wpd)
    features["Image"]["width"], features["Image"]["height"] = image_size
    return features

In [ ]:
features = parse(wpd_data, url)

In [ ]:
doc = posts.find_one(
        {"URL": url}, {"Blocks": 1, "Image": 1, "Text": 1, "URL": 1}
    )